Importações

In [10]:
import pandas as pd
import mca

Leitura dos dados de candidato

In [2]:
tabela_candidato = pd.read_csv('datasets/candidatos.csv', dtype = {
    "ano_eleicao": int, 
    "codigo_eleicao": float,                            #NAN
    "codigo_tipo_eleicao": float,                         #NAN
    "data_eleicao": object, 
    "descricao_eleicao": object, 
    "nome_tipo_eleicao": object, 
    "tipo_abrangencia_eleicao": object, 
    "codigo_totalizacao_turno": float,                    #NAN
    "descricao_totalizacao_turno": object, 
    "numero_turno": int, 
    "descricao_ue": object, 
    "sigla_ue": object, 
    "sigla_uf": object, 
    "codigo_cor_raca": float,                             #NAN
    "codigo_estado_civil": int, 
    "codigo_genero": int, 
    "codigo_grau_instrucao": int, 
    "codigo_municipio_nascimento": int, 
    "codigo_nacionalidade": int, 
    "codigo_ocupacao": int, 
    "cpf": int, 
    "data_nascimento": object, 
    "descricao_cor_raca": object, 
    "descricao_estado_civil": object, 
    "descricao_genero": object, 
    "descricao_grau_instrucao": object, 
    "descricao_nacionalidade": object, 
    "descricao_ocupacao": object, 
    "email": object, 
    "nome": object, 
    "nome_municipio_nascimento": object, 
    "nome_social": object, 
    "sigla_uf_nascimento": object, 
    "titulo_eleitoral": float,                            #NAN
    "codigo_legenda": float,                              #NAN
    "composicao_legenda": object, 
    "nome_legenda": object, 
    "nome_partido": object, 
    "numero_partido": int, 
    "sigla_legenda": object, 
    "sigla_partido": object, 
    "tipo_agremiacao": object, 
    "codigo_cargo": int, 
    "codigo_detalhe_situacao_candidatura": float,         #NAN
    "codigo_situacao_candidatura": int, 
    "concorre_reeleicao": object, 
    "declara_bens": object, 
    "descricao_cargo": object, 
    "descricao_detalhe_situacao_candidatura": object, 
    "descricao_situacao_candidatura": object, 
    "despesa_maxima_campanha": float, 
    "idade_data_eleicao": float,                          #NAN
    "idade_data_posse": float,                            #NAN
    "nome_urna": object, 
    "numero_processo_candidatura": object,
    "numero_protocolo_candidatura": float,                #NAN
    "numero_sequencial": int, 
    "numero_urna": float,                                 #NAN 
    "pergunta": object})

In [3]:
tabela_candidato.head()

,ano_eleicao,codigo_eleicao,codigo_tipo_eleicao,data_eleicao,descricao_eleicao,nome_tipo_eleicao,tipo_abrangencia_eleicao,codigo_totalizacao_turno,descricao_totalizacao_turno,numero_turno,...,descricao_situacao_candidatura,despesa_maxima_campanha,idade_data_eleicao,idade_data_posse,nome_urna,numero_processo_candidatura,numero_protocolo_candidatura,numero_sequencial,numero_urna,pergunta
0,1996,NaN,NaN,NaN,ELEICOES 1996,NaN,NaN,4.0,NAO ELEITO,1,...,NaN,-1.0,-1.0,NaN,NaN,NaN,NaN,-1,11.0,NaN
1,1996,NaN,NaN,NaN,ELEICOES 1996,NaN,NaN,4.0,NAO ELEITO,1,...,NaN,-1.0,-1.0,NaN,NaN,NaN,NaN,-1,13.0,NaN
2,1996,NaN,NaN,NaN,ELEICOES 1996,NaN,NaN,1.0,ELEITO,1,...,NaN,-1.0,-1.0,NaN,NaN,NaN,NaN,-1,15.0,NaN
3,1996,NaN,NaN,NaN,ELEICOES 1996,NaN,NaN,NaN,NaN,1,...,NaN,-1.0,-1.0,NaN,NaN,NaN,NaN,-1,111.0,NaN
4,1996,NaN,NaN,NaN,ELEICOES 1996,NaN,NaN,NaN,NaN,1,...,NaN,-1.0,-1.0,NaN,NaN,NaN,NaN,-1,151.0,NaN


Listagem dos pleitos presentes retorna 254 eleições, dentre 7 eleições ordinárias, 104 suplementares e 143 não informadas

In [4]:
tabela_candidato\
    .fillna(-9999)\
    .groupby([
        'codigo_eleicao', 'ano_eleicao', 'data_eleicao', 'descricao_eleicao', 
        'nome_tipo_eleicao', 'tipo_abrangencia_eleicao'
    ])\
    .size()\
    .reset_index()\
    .rename(columns={0:'count'})

,codigo_eleicao,ano_eleicao,data_eleicao,descricao_eleicao,nome_tipo_eleicao,tipo_abrangencia_eleicao,count
0,-9999.0,1996,-9999,ELEICOES 1996,-9999,-9999,10356
1,-9999.0,1998,-9999,ELEICOES 1998,-9999,-9999,15178
2,-9999.0,2000,-9999,ELEICOES 2000,-9999,-9999,403784
3,-9999.0,2002,-9999,ELEICOES 2002,-9999,-9999,19901
4,-9999.0,2004,-9999,ELEICOES 2004,-9999,-9999,401789
5,-9999.0,2006,-9999,ELEICOES 2006,-9999,-9999,20790
6,-9999.0,2008,-9999,-9999,-9999,-9999,22260
7,-9999.0,2008,-9999,ELEICOES 2008,-9999,-9999,361488
8,-9999.0,2008,-9999,ELEICOES SUPLEMENTARES,-9999,-9999,8
9,-9999.0,2008,-9999,ELEICOES SUPLEMENTARES 2008,-9999,-9999,772


In [5]:
tabela_candidato\
    .fillna(-9999)\
    .groupby(['codigo_tipo_eleicao', 'nome_tipo_eleicao'])\
    .size()\
    .reset_index()\
    .rename(columns={0:'count'})

,codigo_tipo_eleicao,nome_tipo_eleicao,count
0,-9999.0,-9999,1762732
1,1.0,ELEICAO SUPLEMENTAR,695
2,2.0,ELEICAO ORDINARIA,552490


In [6]:
tabela_candidato\
    .loc[tabela_candidato['codigo_tipo_eleicao'] == 2.0]\
    .fillna(-9999)\
    .groupby([
        'codigo_eleicao', 'ano_eleicao', 'data_eleicao', 'descricao_eleicao', 
        'nome_tipo_eleicao', 'tipo_abrangencia_eleicao'
    ])\
    .size()\
    .reset_index()\
    .rename(columns={0:'count'})

,codigo_eleicao,ano_eleicao,data_eleicao,descricao_eleicao,nome_tipo_eleicao,tipo_abrangencia_eleicao,count
0,143.0,2014,05/10/2014,ELEICOES GERAIS 2014,ELEICAO ORDINARIA,FEDERAL,26162
1,144.0,2014,26/10/2014,ELEICOES GERAIS 2014,ELEICAO ORDINARIA,FEDERAL,60
2,220.0,2016,02/10/2016,ELEICOES MUNICIPAIS 2016,ELEICAO ORDINARIA,MUNICIPAL,496895
3,221.0,2016,30/10/2016,ELEICOES MUNICIPAIS 2016,ELEICAO ORDINARIA,MUNICIPAL,228
4,295.0,2018,07/10/2018,ELEICAO GERAL FEDERAL 2018,ELEICAO ORDINARIA,FEDERAL,28
5,296.0,2018,28/10/2018,ELEICAO GERAL FEDERAL 2018,ELEICAO ORDINARIA,FEDERAL,4
6,297.0,2018,07/10/2018,ELEICOES GERAIS ESTADUAIS 2018,ELEICAO ORDINARIA,ESTADUAL,29057
7,298.0,2018,28/10/2018,ELEICOES GERAIS ESTADUAIS 2018,ELEICAO ORDINARIA,ESTADUAL,56


Considerando a quantidade de dados razoável e a relativa abundância de informações, optou-se por dar foco apenas nas eleições ordinárias. Além disso, foram removidas as colunas julgadas irrelevantes: codigo_tipo_eleicao, nome_tipo_eleicao, email, cpf, numero_processo_candidatura, numero_protocolo_candidatura, numero_sequencial e pergunta.<br>
Além disso, removeremos todos os candidatos cuja candidatura foi indeferida, bem como as colunas vinculadas a tal: codigo_detalhe_situacao_candidatura, codigo_situacao_candidatura, descricao_detalhe_situacao_candidatura e descricao_situacao_candidatura

In [7]:
df_ordinarias = tabela_candidato.loc[
    (tabela_candidato['codigo_tipo_eleicao'] == 2.0) &
    (tabela_candidato['codigo_situacao_candidatura'] == 12)
]
df_work = df_ordinarias.drop([
    "codigo_tipo_eleicao", 
    "nome_tipo_eleicao", 
    "email",
    "cpf",
    "numero_processo_candidatura", 
    "numero_protocolo_candidatura", 
    "numero_sequencial",
    "pergunta",
    "codigo_detalhe_situacao_candidatura", 
    "codigo_situacao_candidatura",
    "descricao_detalhe_situacao_candidatura", 
    "descricao_situacao_candidatura"
], axis=1)
df_work.head()

,ano_eleicao,codigo_eleicao,data_eleicao,descricao_eleicao,tipo_abrangencia_eleicao,codigo_totalizacao_turno,descricao_totalizacao_turno,numero_turno,descricao_ue,sigla_ue,...,tipo_agremiacao,codigo_cargo,concorre_reeleicao,declara_bens,descricao_cargo,despesa_maxima_campanha,idade_data_eleicao,idade_data_posse,nome_urna,numero_urna
1762732,2014,143.0,05/10/2014,ELEICOES GERAIS 2014,FEDERAL,5.0,SUPLENTE,1,ACRE,AC,...,COLIGACAO,7,N,N,DEPUTADO ESTADUAL,400000.0,NaN,45.0,CHIQUINHO DIAS,27147.0
1762733,2014,143.0,05/10/2014,ELEICOES GERAIS 2014,FEDERAL,5.0,SUPLENTE,1,ACRE,AC,...,COLIGACAO,7,N,S,DEPUTADO ESTADUAL,1000000.0,NaN,50.0,FERNANDO CARDOSO,23414.0
1762734,2014,143.0,05/10/2014,ELEICOES GERAIS 2014,FEDERAL,5.0,SUPLENTE,1,ACRE,AC,...,COLIGACAO,7,N,S,DEPUTADO ESTADUAL,690000.0,NaN,48.0,DR. JAKSON RAMOS,13777.0
1762735,2014,143.0,05/10/2014,ELEICOES GERAIS 2014,FEDERAL,5.0,SUPLENTE,1,ACRE,AC,...,COLIGACAO,7,N,S,DEPUTADO ESTADUAL,500000.0,NaN,39.0,ROMARIO TAVARES,45122.0
1762736,2014,143.0,05/10/2014,ELEICOES GERAIS 2014,FEDERAL,5.0,SUPLENTE,1,ACRE,AC,...,COLIGACAO,7,N,N,DEPUTADO ESTADUAL,500000.0,NaN,40.0,DR. JACSON,10222.0


Iniciaremos o estudo com as eleições para deputado estadual/distrital em 2018 (por possuirem mais dados)

In [8]:
df_deputado_2018 = df_work\
            .loc[(df_work['codigo_eleicao'] == 297.0) & (df_work['codigo_cargo'].isin([7, 8]))]\
            .drop([
                'ano_eleicao', 'data_eleicao', 'descricao_eleicao', 'descricao_cargo',
                'tipo_abrangencia_eleicao', 'codigo_eleicao', 'codigo_cargo'
            ], axis=1)
df_deputado_2018.head()

,codigo_totalizacao_turno,descricao_totalizacao_turno,numero_turno,descricao_ue,sigla_ue,sigla_uf,codigo_cor_raca,codigo_estado_civil,codigo_genero,codigo_grau_instrucao,...,sigla_legenda,sigla_partido,tipo_agremiacao,concorre_reeleicao,declara_bens,despesa_maxima_campanha,idade_data_eleicao,idade_data_posse,nome_urna,numero_urna
2286773,5.0,SUPLENTE,1,ACRE,AC,AC,5.0,1,2,7,...,NaN,PSOL,COLIGACAO,N,N,0.0,NaN,30.0,RONY CARTOLIN,50000.0
2286774,5.0,SUPLENTE,1,ACRE,AC,AC,1.0,1,4,6,...,NaN,PTB,PARTIDO ISOLADO,N,N,0.0,NaN,39.0,ANDREA PUPIO,14222.0
2286777,2.0,ELEITO POR QP,1,ACRE,AC,AC,3.0,1,2,6,...,NaN,PODE,COLIGACAO,S,S,0.0,NaN,45.0,JOSA DA FARMACIA,19019.0
2286778,5.0,SUPLENTE,1,ACRE,AC,AC,3.0,3,2,7,...,NaN,DEM,COLIGACAO,N,S,0.0,NaN,64.0,RUBENS MARTINS,25128.0
2286781,5.0,SUPLENTE,1,ACRE,AC,AC,3.0,3,4,8,...,NaN,PP,COLIGACAO,N,S,0.0,NaN,58.0,LUCILA,11456.0


In [9]:
df_deputado_2018\
    .fillna(-9999)\
    .groupby(['codigo_totalizacao_turno', 'descricao_totalizacao_turno'])\
    .size()\
    .reset_index()\
    .rename(columns={0:'count'})

,codigo_totalizacao_turno,descricao_totalizacao_turno,count
0,2.0,ELEITO POR QP,888
1,3.0,ELEITO POR MEDIA,171
2,4.0,NAO ELEITO,2304
3,5.0,SUPLENTE,13996


In [11]:
df_dep_est_18_mca = mca.MCA(df_deputado_2018.drop(['codigo_totalizacao_turno', 'descricao_totalizacao_turno'], axis=1))
df_dep_est_18_mca.head()

TypeError: ufunc 'add' did not contain a loop with signature matching types dtype('<U174890') dtype('<U174890') dtype('<U174890')